In [7]:
import yfinance as yf
from datetime import date
import pandas as pd

In [8]:
def GetQuote(ticker="GME",start = "2018-01-01" ,end=str(date.today())):
    """
    """
    if not isinstance(ticker,str):
        raise Exception('String input expected')
    if not isinstance(start,str):
        raise Exception('String input expected')
    if not isinstance(end,str):
        raise Exception('String input expected')
        
    df = yf.download(ticker, start, end)
    
    return df

In [24]:
isinstance(df,pd.core.frame.DataFrame)

True

In [25]:
def RelativeStrengthIndex(data, num):
    
    """
    """
    
    if not isinstance(data,pd.core.frame.DataFrame):
        raise Exception('DataFrame input expected')
    if not isinstance(num, int):
        raise Exception('Integer input expected')
    if num < 7 or num > 21:
        raise Exception('Unusual numeric input detected')
    if (num > len(data)):
        raise Exception('Insufficient data for calculation')
        

    data_keys=list(data.index)
    data_list=list(data['Adj Close'])
    
    
    result = {}
    last_price = -1
    gains_losses_list = []
    for x in range(len(data_list)):
        if (last_price != -1):
            diff = round((data_list[x] - last_price), 2)
            
            if (diff > 0):
                gains_losses = [ data_list[x], diff, 0 ]
            elif (diff < 0):
                gains_losses = [ data_list[x], 0, abs(diff) ]
            else:
                gains_losses = [ data_list[x], 0, 0 ]
            
            gains_losses_list.append(gains_losses)
        sum_gains = 0
        sum_losses = 0
        avg_gains = 0
        avg_losses = 0 
        if (x == num):
            series = gains_losses_list[-num::]
        
            for y in series:
                sum_gains += y[1]
                sum_losses += y[2]
            avg_gains = sum_gains / num
            avg_losses = sum_losses / num
            rs = avg_gains / avg_losses
            rsi = 100 - (100 / (1 + rs))
            last_gain_avg = avg_gains
            last_loss_avg = avg_losses
            result[data_keys[x]] = round(rsi, 2)
        if (x > num):
            current_list = gains_losses_list[-1::]
            current_gain = current_list[0][1]
            current_loss = current_list[0][2]
            current_gains_avg = (last_gain_avg * (num - 1) + current_gain) / num
            current_losses_avg = (last_loss_avg * (num - 1) + current_loss) / num
            rs = current_gains_avg / current_losses_avg
            rsi = 100 - (100 / (1 + rs))
            last_gain_avg = current_gains_avg
            last_loss_avg = current_losses_avg
            result[data_keys[x]] = round(rsi, 2)    
      
        last_price = data_list[x]
        
    return result

In [26]:
def macdIndex(data,shortSpan=12,longSpan=26,singalSpan=9):
    
    """
    
    """
    #Calculate the MACD and Signal Line indicators
    #Calculate the Short Term Exponential Moving Average
    ShortEMA = data.Close.ewm(span=shortSpan, adjust=False).mean() #AKA Fast moving average
    #Calculate the Long Term Exponential Moving Average
    LongEMA = data.Close.ewm(span=longSpan, adjust=False).mean() #AKA Slow moving average
    #Calculate the Moving Average Convergence/Divergence (MACD)
    MACD = ShortEMA - LongEMA
    #Calcualte the signal line (exp weighted)
    signal = MACD.ewm(span=singalSpan, adjust=False).mean()
    
    return signal,MACD

In [29]:
df = GetQuote()
# RelativeStrengthIndex(df, 14)
macdIndex(df)

[*********************100%***********************]  1 of 1 downloaded


(Date
 2018-01-02     0.000000
 2018-01-03    -0.000957
 2018-01-04    -0.000548
 2018-01-05     0.006442
 2018-01-08     0.025950
 2018-01-09     0.057328
 2018-01-10     0.096443
 2018-01-11     0.142606
 2018-01-12     0.155342
 2018-01-16     0.136531
 2018-01-17     0.104047
 2018-01-18     0.070202
 2018-01-19     0.041427
 2018-01-22     0.007989
 2018-01-23    -0.029030
 2018-01-24    -0.072987
 2018-01-25    -0.121057
 2018-01-26    -0.165397
 2018-01-29    -0.203263
 2018-01-30    -0.235900
 2018-01-31    -0.267158
 2018-02-01    -0.297698
 2018-02-02    -0.332368
 2018-02-05    -0.376307
 2018-02-06    -0.415075
 2018-02-07    -0.442360
 2018-02-08    -0.470219
 2018-02-09    -0.490481
 2018-02-12    -0.509706
 2018-02-13    -0.529619
                 ...    
 2021-03-08     8.960859
 2021-03-09    12.905934
 2021-03-10    17.654373
 2021-03-11    22.537414
 2021-03-12    27.269271
 2021-03-15    30.888428
 2021-03-16    33.359067
 2021-03-17    34.930457
 2021-03-18    35.6

In [19]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,17.96,18.29,17.78,18.26,15.95,2832700
2018-01-03,18.29,18.37,17.92,18.20,15.90,3789200
2018-01-04,18.20,18.38,17.96,18.32,16.01,2781300
2018-01-05,18.38,18.73,18.22,18.68,16.32,3019000
2018-01-08,18.80,19.40,18.80,19.23,16.80,3668400
2018-01-09,19.31,19.75,19.31,19.55,17.08,3662600
2018-01-10,19.39,19.92,19.33,19.68,17.19,4012800
2018-01-11,19.92,20.31,19.77,19.96,17.44,4213100
2018-01-12,18.50,18.54,17.63,17.76,15.52,15445800
